# Imports
All problem-speciffic files (code) in in the folder `HyperElasticity3D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [ ]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from HyperElasticity3D.jax_energy import energy_jax
from HyperElasticity3D.mesh import MeshHyperElasticity3D
from HyperElasticity3D.rotate_boundary import rotate_boundary

import numpy as np
from jax import config

config.update("jax_enable_x64", True)

# Setup of the problem

In [ ]:
rotation_per_iter = 4 * 2 * np.pi / 24
mesh = MeshHyperElasticity3D(mesh_level=3)
params, adjacency, u_init = mesh.get_data_jax()
energy_hyperelasticity = EnergyDerivator(energy_jax, params, adjacency, u_init)

params = rotate_boundary(params, angle=rotation_per_iter)
energy_hyperelasticity.params = params

F, dF, ddF = energy_hyperelasticity.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", elastic_kernel=mesh.elastic_kernel, verbose=True, tol=1e-3)

# Solution via Newton's method

In [ ]:
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-4, linesearch_tol=1e-3)
res

### And other stepps of rotation will be as follows:

In [ ]:
# Apply timestep = rotation of the right boundary
params = rotate_boundary(params, angle=rotation_per_iter)
energy_hyperelasticity.params = params

# Get derivatives
F, dF, ddF = energy_hyperelasticity.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", elastic_kernel=mesh.elastic_kernel, verbose=True, tol=1e-3)
# Actualize initial guess
u_init = res["x"]

# Newton method
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-4, linesearch_tol=1e-3)
res